<a href="https://colab.research.google.com/github/ASR-X/ManuScripted/blob/main/backend/Essay_Scorer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install Dependencies

In [ ]:
#@title Run this cell for all initialization procedures (run in google colab!)
#capture
!pip install fastapi nest-asyncio pyngrok uvicorn
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip /content/ngrok-stable-linux-amd64.zip
#!git clone https://github.com/ASR-X/packhacks.git
!./ngrok authtoken 1q4QJih5O9lXHzFzujsNZ5a46Wz_xh5a7kjfxAuXNuV5VxSL
!pip install -q matplotlib numpy pandas pathlib seaborn
!pip install -q tensorflow 
!pip install -q git+https://github.com/tensorflow/docs
!git clone https://github.com/user501254/regression-aes.git
!pip install -r /content/regression-aes/requirements.txt
!sudo apt install openjdk-8-jdk
!sudo update-alternatives --set java /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java
!pip install language-check
!pip install pycontractions
!pip install textstat
from __future__ import absolute_import, division, print_function, unicode_literals
import itertools
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.metrics import cohen_kappa_score
import tensorflow as tf
import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling
from tensorflow import keras
from tensorflow.keras import layers
import math
print(tf.__version__)
from __future__ import absolute_import, division, print_function, unicode_literals
import pathlib
from collections import Counter
from statistics import mean
import language_check
import matplotlib.pyplot as plt
import nltk
import numpy as np
import pandas as pd
import seaborn as sns; 
import textstat as ts
from textblob import TextBlob
from textblob import Word
nltk.download('punkt'); nltk.download('brown'); nltk.download('averaged_perceptron_tagger')
sns.set(style="ticks", color_codes=True)
dataset = pd.read_pickle('/content/regression-aes/output/training_set_rel3.pkl')
dataset
# dataset = dataset.dropna(axis='columns').drop(columns=['essay', 'essay_set'])
dataset = dataset.dropna(axis='columns').drop(columns=['essay'])
train_dataset = dataset.sample(frac=0.8,random_state=0)
test_dataset = dataset.drop(train_dataset.index)
train_stats = train_dataset.describe().pop("domain1_score").transpose()
train_labels = train_dataset.pop('domain1_score')
test_labels = test_dataset.pop('domain1_score')
def norm(x):
    return (x - train_stats['mean']) / train_stats['std']
normed_train_data = norm(train_dataset)
normed_test_data = norm(test_dataset)

def build_model():
    model = keras.Sequential([
    layers.Dense(16, activation='swish', input_shape=[len(train_dataset.keys())]),
    layers.BatchNormalization(),
    layers.Dense(16, activation='swish'),
    layers.Dense(1)
  ])

    optimizer = tf.keras.optimizers.RMSprop(0.001)

    model.compile(loss='mse',
                optimizer=optimizer,
                metrics=['mae', 'mse'])
    return model
model = build_model()
model.load_weights("bruh.h5")
def add_meta_feature_columns(index, df, blob):
    
    # Essay Length (number of words)
    df.at[index, 'essay_length'] = len(blob.words)

    # Average Sentence Length
    sentence_lengths = [len(sentence.split(' ')) for sentence in blob.sentences]
    df.at[index, 'avg_sentence_length'] = mean(sentence_lengths)

    # Average Word Length
    word_lengths = [len(word) for word in blob.words]
    df.at[index, 'avg_word_length'] = mean(word_lengths)

    # Sentiment
    df.at[index, 'sentiment'] = blob.sentiment.polarity
def add_grammar_feature_columns(index, df, blob, essay):
    
    # Number of noun phrases
    df.at[index, 'noun_phrases'] = len(blob.noun_phrases)

    # Number of possible spelling and grammatical Mistakes
    # Takes significant amount of time
    print("Processed %5d essays for correctness..." % (index + 1), end="\r")
    languageTool = language_check.LanguageTool('en-US')
    df.at[index, 'syntax_errors'] = len(languageTool.check(essay))
def add_redability_feature_columns(index, df, essay):
    
    # Number of noun phrases
    df.at[index, 'readability_index'] = ts.automated_readability_index(essay)

    df.at[index, 'difficult_words'] = ts.difficult_words(essay)

def getPred(essay):
  d = {'essay_set': [0], 'essay_length': [40.018926], 'avg_sentence_length': [1.217917]
      , 'avg_word_length': [-0.50648], 'sentiment': [-0.784432]
      , 'noun_phrases': [1.831376], 'syntax_errors': [0.256830]
      , 'readability_index': [-0.035585], 'difficult_words': [3.068520]}
  df = pd.DataFrame(data=d,index=[1000])
  blob = TextBlob(essay)
  add_meta_feature_columns(1000, df, blob)
  add_grammar_feature_columns(1000, df, blob, essay)
  add_redability_feature_columns(1000, df, essay)
  dg=df

  df=norm(df)
  for k in df.index:
    df["essay_length"][k]=df["essay_length"][k]*60
    df["syntax_errors"][k]=df["syntax_errors"][k]*2

  def sigmoid(x):
    return 1 / (1 + math.exp(-x*3))
  a=sigmoid(model.predict(df)[0][0]*2-1)*100
  print(a)

  b=str(f'{a:.20f}')[:4]
  dg["score"]=b
  out=dg.to_dict('records')[0]
  out.pop("essay_set")
  out["essaylength"] = out.pop("essay_length")
  out["avgsentencelength"] = out.pop("avg_sentence_length")
  out["avgwordlength"] = out.pop("avg_word_length")
  out["nounphrases"] = out.pop("noun_phrases")
  out["syntaxerrors"] = out.pop("syntax_errors")
  out["readabilityindex"] = out.pop("readability_index")
  out["difficultwords"] = out.pop("difficult_words")
  out={k:str(v)[:5] for k,v in out.items()}

  return out


     |████████████████████████████████| 51kB 2.7MB/s 
     |████████████████████████████████| 747kB 7.4MB/s 
     |████████████████████████████████| 51kB 6.0MB/s 
     |████████████████████████████████| 61kB 6.7MB/s 
     |████████████████████████████████| 10.1MB 28.5MB/s 
     |████████████████████████████████| 61kB 7.4MB/s 
  Created wheel for pyngrok: filename=pyngrok-5.0.4-cp37-none-any.whl size=18971 sha256=d85cff5ac968b682ed48d7f7df506ab45d938bd8acef17ee6319d719ae2bfb40
  Stored in directory: /root/.cache/pip/wheels/8a/82/b1/cecfba4ff6e2f05777a5a4a65b46c1114842453d5a0e61bdd4
Successfully built pyngrok
--2021-03-21 16:31:15--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 34.226.165.133, 52.204.244.158, 34.196.154.11, ...
Connecting to bin.equinox.io (bin.equinox.io)|34.226.165.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13773305 (13M) [application/octet-stream]
Saving to: ‘ngrok-sta

# Run FastAPI server

In [ ]:
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
from fastapi.responses import HTMLResponse
from pydantic import BaseModel

class EssayIn(BaseModel):
    essay : str

class ScoreOut(BaseModel):
    sentiment: str
    score: str
    essaylength: str
    avgsentencelength: str
    avgwordlength: str
    nounphrases: str
    syntaxerrors: str
    readabilityindex: str
    difficultwords: str



app = FastAPI()

app.add_middleware(
    CORSMiddleware,
    allow_origins='*',
    allow_credentials=True,
    allow_methods='*',
    allow_headers='*',
)


@app.api_route("/", methods=["GET", "POST"])
async def root():
    return 'alive'


@app.post("/score", response_model=ScoreOut)
async def rate_essay(essayin: EssayIn):
    ret=getPred(essayin.essay)

    return ret 

In [ ]:
import nest_asyncio
from pyngrok import ngrok
import uvicorn

ngrok_tunnel = ngrok.connect(80, bind_tls=True)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=80)